In [2]:
!pip install sentence_transformers bert_score evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 184.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 84.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00


In [3]:
! pip install rouge_score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=012c63297a0f39ab14f5789e4d2baa1e4d53fc917f2510c7a544e604c9af578e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import os
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on: {device}")
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)
model.to(device)

def load_data(judgement_folder, summary_folder, max_files=6000):
    data = []
    judgement_files = sorted(os.listdir(judgement_folder))[:max_files]
    summary_files = sorted(os.listdir(summary_folder))[:max_files]
    for j_file, s_file in zip(judgement_files, summary_files):
        with open(os.path.join(judgement_folder, j_file), 'r') as j_f:
            judgement_text = j_f.read()
        with open(os.path.join(summary_folder, s_file), 'r') as s_f:
            summary_text = s_f.read()
        data.append({"judgement": judgement_text, "summary": summary_text})
    
    return data
    
test_data = load_data("/kaggle/input/legal-case-document-summarization/dataset/IN-Abs/test-data/judgement", "/kaggle/input/legal-case-document-summarization/dataset/IN-Abs/test-data/summary",max_files=100)
train_data = load_data("/kaggle/input/legal-case-document-summarization/dataset/IN-Abs/train-data/judgement", "/kaggle/input/legal-case-document-summarization/dataset/IN-Abs/train-data/summary")

train_dataset = Dataset.from_list(train_data) 
test_dataset = Dataset.from_list(test_data)

def preprocess_function(examples):
    model_inputs = tokenizer(examples["judgement"], max_length=1024, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=256, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",   # Evaluate every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3, 
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=None,
    save_steps=0,
    report_to="none",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

trainer.train()

model.save_pretrained('./fine_tuned_pegasus_model/model.safetensors')
tokenizer.save_pretrained('./fine_tuned_pegasus_model/tokenizer_config.json')

print("Training complete and model saved.")


Training on: cuda


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn

Epoch,Training Loss,Validation Loss
1,2.270900,2.108260
2,2.167900,2.037503
3,2.113000,2.018830


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default 

Training complete and model saved.


In [ ]:
import os
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import evaluate
from bert_score import score
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()

model = PegasusForConditionalGeneration.from_pretrained("/kaggle/working/fine_tuned_pegasus_model").to(device)
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")

sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

rouge = evaluate.load("rouge")

test_dir = '/kaggle/input/legal-case-document-summarization/dataset/IN-Abs/test-data/judgement'
summary_dir = '/kaggle/input/legal-case-document-summarization/dataset/IN-Abs/test-data/summary'

generated_summaries = []
reference_summaries = []
cosine_similarities = []

for filename in os.listdir(test_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(test_dir, filename), 'r', encoding='utf-8') as file:
            test_document = file.read()

        with open(os.path.join(summary_dir, filename), 'r', encoding='utf-8') as ref_file:
            reference_summary = ref_file.read()
            reference_summaries.append(reference_summary)

        inputs = tokenizer(test_document, return_tensors="pt", max_length=1024, truncation=True)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Generate the summary
        summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=150, min_length=40, 
                                     length_penalty=2.0, num_beams=4, early_stopping=True)

        # Decode the generated summary
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        generated_summaries.append(generated_summary)

        # Calculate cosine similarity using sentence embeddings
        ref_embedding = sentence_model.encode([reference_summary])[0]
        gen_embedding = sentence_model.encode([generated_summary])[0]

        cosine_sim = cosine_similarity([ref_embedding], [gen_embedding])
        cosine_similarities.append(cosine_sim[0][0])

# Calculate average cosine similarity
average_cosine_similarity = np.mean(cosine_similarities)
print(f"Average Cosine Similarity for Pegasus: {average_cosine_similarity}")

# Calculate ROUGE score
rouge_scores = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print(f"ROUGE scores for Pegasus: {rouge_scores}")

# Calculate BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang='en', rescale_with_baseline=True)
avg_f1 = np.mean(F1.numpy())
print(f"BERTScore F1 for Pegasus: {avg_f1}")
